In [1]:
import re, math
import numpy as np

import nltk
nltk.download('wordnet') 
from nltk.corpus import wordnet

import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.autograd import Variable
import torch.nn.functional as F 
import import_ipynb
from MoveData import Options, json2datatools, num_batches, nopeak_mask, create_masks
from EncoderDecoder import Encoder, Decoder

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:777)>


importing Jupyter notebook from MoveData.ipynb
importing Jupyter notebook from EncoderDecoder.ipynb
importing Jupyter notebook from Elements.ipynb


## Neural Network as an Agent

In this section we will be putting everything together. We learned AI to make AI robots so let finally do that. 

If you are not using this notebook to learn, change the below variable `teaching` to `False` so that other notebooks can import the functions defined in this notebook without running all the examples, if you are here to learn and interact with the notebook, change it to `True`

The cell below you have seen before, we will need the input and output vocabulary fields `infield, outfield` for our demonstration of how a sequence of words is represented by the transformer and the role that probability plays in the model's outputs. Note, for the time being, we are assuming that the vocabulary consists of all the words in the training set, nothing more nothing less, so if since running START_HERE you have added some lines of data, simply retrain the model from START_HERE before moving on

In [2]:
teaching = True

opt = Options(batchsize=2, device = torch.device("cpu"), epochs=25, lr=0.01, 
              beam_width=3, max_len = 25, save_path = '../saved/weights/model_weights')

data_iter, infield, outfield, opt = json2datatools(path='../saved/pairs.json', opt=opt)

Until we built more capabilities into chloe, we will have to reply on a few software tools. The first tool is a tool for expanding chloe's vocabulary without having to learn them from scratch.

nltk is the [Natural Language Toolkit](https://www.nltk.org/) that we will be using for things such as synonym matching, that way when you say "adore", Chloe knows it means the same thing as "like", even if "adore" is not in Chloe's vocabulary. 

Run the cell below to see how we go from string words to integers and how when we cannot find a word in our vocabulary, we try to find a synonym for that word that is in our vocabulary. 

In [3]:
def get_synonym(word, field, explain=False):
    syns = wordnet.synsets(word)
    for s in syns:
        if explain: print('synonym:', s.name())
        for l in s.lemmas():
            if explain: print('-lemma:', l.name())
            if field.vocab.stoi[l.name()] != 0:
                if explain: print('found in vocab', l.name())
                return field.vocab.stoi[l.name()]
    return 0 # if we cannot find a synonym, return 0

In [4]:
if teaching:
    print('token = ', get_synonym("fine", infield, explain=True))

synonym: fine.n.01
-lemma: fine
-lemma: mulct
-lemma: amercement
synonym: ticket.v.01
-lemma: ticket
-lemma: fine
synonym: all_right.s.01
-lemma: all_right
-lemma: fine
-lemma: o.k.
-lemma: ok
found in vocab ok
token =  6


The next function takes your sentence in the form of text and converts it to a sequence of tokens within a torch tensor

In [5]:
def string2tensor(string, inputfield, explain=False):
    '''
    input:
        string (str) input sentence
        inputfield a PyTorch torchtext.data.Field object
        explain, set this to True if you want to see how the sentence was split 
    output:
        sequence of tokens (torch tensor of integers) shape  
    '''
    sentence = inputfield.preprocess(string)
    if explain: print(sentence)
    integer_sequence = []
    for tok in sentence:
        if inputfield.vocab.stoi[tok] != 0:
            integer_sequence.append(inputfield.vocab.stoi[tok])
        else:
            integer_sequence.append(get_synonym(tok, inputfield))
    return torch.LongTensor([integer_sequence])

In [6]:
input_sequence = string2tensor("ok, aren't -you [a robot?", infield, explain=True)
print(input_sequence)

['ok', ',', "aren't", 'you', 'a', 'robot', '?']
tensor([[ 6,  0,  0,  3,  9, 31,  2]])


Lets assemble the Encoder and Decoder into the Transformer, like older sequence to sequence models, Transformers also encode the sentence into vector representations and pass those representations along to the decoder to generate the response/reply/output/translation/etc. The Encoder and Decoder we can use separately such as `model.encoder(arguments)`. The very last part of the transformer is the mapping of each vector in the decoder output to logits for each token in the output vocabulary `output = self.out(d_output)`. There are as many logits as there are tokens in the output vocabulary. In the conceptual diagram we pretend that each decoder output is represented with 4 dimensional vectors and the last linear layer maps this vector to the output vocabulary which is only 5 tokens including the end of sentence `<eos>` token. In a more realistic model, the decoder might output 512 dimensional vectors and the last linear layer maps this vector to the output vocabulary which is a few thousand tokens wide, including words, punctuation marks and the end of sentence, unknown etc tokens. The softmax later on will balance all these logits to sum to 1.0 so that you can treat this as a probability distribution over the vocabulary from which the agent will draw/sample its next word. 

<img src="../saved/images/vec2vocab.png" height=500 width=600>

Define the Transformer class, instantiate a model and load the weights you trained in START_HERE into that model by running the next 2 cells. 

In [7]:
class Transformer(nn.Module):
    def __init__(self, in_vocab_size, out_vocab_size, emb_dim, n_layers, heads, dropout):
        super().__init__()
        self.encoder = Encoder(in_vocab_size, emb_dim, n_layers, heads, dropout)
        self.decoder = Decoder(out_vocab_size, emb_dim, n_layers, heads, dropout)
        self.out = nn.Linear(emb_dim, out_vocab_size)
    def forward(self, src_seq, trg_seq, src_mask, trg_mask):
        e_output = self.encoder(src_seq, src_mask)
        d_output = self.decoder(trg_seq, e_output, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [8]:
emb_dim, n_layers, heads, dropout = 32, 3, 8, 0.01 
chloe = Transformer(len(infield.vocab), len(outfield.vocab), emb_dim, n_layers, heads, dropout)
chloe.load_state_dict(torch.load(opt.save_path))

<All keys matched successfully>

The way we use the transformer is that we first use the encoder to modify the input sequence into a new sequence of vectors where each vector uses the other vectors in the sentence as context to re-represented itself. 

` encoding = model.encoder(input_sequence, input_mask)`

Then we run a loop. The loop will keep running until it gets to `max_len` or until it believes it is finished with it's sentence. 

To get started talking, the decoder will need two sequences. The sentence it is responding to `encoding` and a start token.

`init_tok = outfield.vocab.stoi['<sos>']` 

`decoder_input = torch.LongTensor([[init_tok]])` 

Then the loops starts. Each cycle the decoder takes the decoder input, which are the words spoken thus far, and outputs the next word for each of those decoder input tokens. For example, if you input `<sos>`, you get `hi`.

`<sos>` -> `hi`

`<sos> hi` -> `hi I`

`<sos> hi I` -> `hi I am`

`<sos> hi I am` -> `hi I am chloe`

`<sos> hi I am chloe` -> `hi I am chloe <eos>`

Lets take a look at the second step to see how we got `<sos> hi` -> `hi I`. The decoder input of 2 vectors goes in and 2 vectors come out. Those two vectors are a shift to the right, or a shift into the future by one step, thus `<sos>` again produces `hi` and `hi` produces `I`. 

Actually, when `<sos>`, the vector, is inputted to the decoder, the decoder takes this vector and using `encoding` as the context, produces a vector that is the size of the vocabulary. The code `self.out = nn.Linear(emb_dim, out_vocab_size)` shows you that `output = self.out(d_output)` is tranforming a vector of size `emb_dim` and expanding that out to size `out_vocab_size`. 

`softout = F.softmax(out, dim=-1)` takes the model output (the logits) and balances/smoothens the these numbers such that they add up to 1.0 like a real distribution. The distribution `distr = Categorical(probs=softout)` is over every token in the output vocabulary.  In this line:

`action = distr.sample()[:,-1].unsqueeze(0)`

We pick a word from the output vocabulary while giving words with more larger more positive logits a higher chance of being picked, aka sampling from the distribution. If you have not taken probability theory, this subject is considered a pre-requisite for machine learning, but I dont think it should stop you from understanding this lesson. Imagine we are having a raffle. The logits are like how many raffle tickets you have. If you are a word token with twice as many raffle tickets as me, you have about double the chance of being picked than I do, but I might still be picked because its random. 

`[:,-1]` is just saying we are only going to take the new output, the output of the leading token, For `<sos> hi` that would be the word `hi` whose output is `I`. `.unsqueeze(0)` adds the batch dimension. `decoder_input = torch.cat((decoder_input, action), dim=1)` appends the next token to the sequence we are building so that the next input to the decoder is `<sos> hi I`

`de_str = ' '.join([outfield.vocab.itos[tok] for tok in decoder_input[0][1:-1]])` joins the tokens in their string form together, separated by spaces `' '`, to give us our string readable output response

In [21]:
def talk_to_chloe(input_str, model, opt, infield, outfield):
    '''
    input:
        input_str is a string, it is what you want to say to the dialogue model
        model is a Transformer model with encoder, decoder and a last layer linear transformation
        opt is an options object with the maximum length of the output sequence opt.max_len
        infield and outfield are the data.fields that store the vocabulary
    output:
        an output string response from the dialogue model
    
    Note: this version assumes we are evaluating the model on CPU 
    '''
    model.eval()
    model.cpu()
    input_sequence = string2tensor(input_str, infield) # string to tensor 
    input_mask = (input_sequence != infield.vocab.stoi['<pad>']).unsqueeze(-2) #make input mask
    encoding = model.encoder(input_sequence, input_mask) # use the encoder rerepresent the input
    init_tok = outfield.vocab.stoi['<sos>'] # this is the integer for the start token
    decoder_input = torch.LongTensor([[init_tok]]) # use start token to initiate the decoder
    
    # continue obtaining the next decoder token until decoder outputs and end token or til max_len 
    for pos in range(opt.max_len):
        decoder_input_mask = nopeak_mask(size=pos+1, opt=opt) # make target mask
        # the out vector contains the logits that are rebalanced by the softmax
        out = model.out(model.decoder(decoder_input, encoding, input_mask, decoder_input_mask))
        softout = F.softmax(out, dim=-1) 
        #softout is a categorical probability distribution over the output vocab
        distr = Categorical(probs=softout)
        action = distr.sample()[:,-1].unsqueeze(0) # sample from that distribution to get next token
        # concatenate that token to our running list of output tokens 
        decoder_input = torch.cat((decoder_input, action), dim=1) 
        # if the model outputs an end of sentence token, it is done with this sentence
        if outfield.vocab.itos[action] == '<eos>':
            # [1:-1] excludes the start and end token from the output string 
            de_str = ' '.join([outfield.vocab.itos[tok] for tok in decoder_input[0][1:-1]])
            return de_str
        
    de_str = ' '.join([outfield.vocab.itos[tok] for tok in decoder_input[0]])
    return de_str

In [22]:
talk_to_chloe("how?", chloe, opt, infield, outfield)

'meowci beaucoup'

<img src="https://media2.giphy.com/media/2Faz8bcnGfXVrV2ZG/source.gif">

## How can I help you or get help from you?

[Support *ChloeRobotics* on Patreon and send us a message](https://www.patreon.com/chloerobotics)

## Questions?

email chloe.the.robot [at] gmail [dot] com 